In [3]:
import os
os.getcwd()

'/Users/jtbbl/Library/CloudStorage/OneDrive-UniversityofBirmingham/MSc Health Data Science/Mod 6 - Multi-Model/Presentation'

In [4]:
import numpy as np
import pandas as pd

# Reading in data

In [5]:
# Reading data in
raw_multi_model_df = pd.read_excel('Group Project v3.xlsx', sheet_name='Data', index_col='Sample')

# Splitting data 

In [6]:
# Splitting data into the 3 datasets
clinical_df = raw_multi_model_df.iloc[:,0:11]
glycome_df = raw_multi_model_df.loc[:, 'Low-branching glycans (LB)':'Antennary fucosylation (AF)']
scfa_df = raw_multi_model_df.loc[:, '2-hydroxybutyrate':'Valerate']

# Data cleaning

In [7]:
# Data cleaning - Clinical data
clinical_df_clean = clinical_df.copy()

# Removing leading white space in Diarrhea
clinical_df_clean['Diarrheal/ Non Diarrheal']=[i.strip() for i in clinical_df_clean['Diarrheal/ Non Diarrheal']]

# A sample is 2 years old but cohort study (5 samples under 18)
clinical_df_clean['Age'][clinical_df_clean['Age']<18]=np.nan

# Gender/ Age/ Toilet/ Hand soap/ Reason for hospital admission/Co-morbidities/Antibiotics (Y/N)
# - (missing values for same person - sample 199)
clinical_df_clean['Gender'].describe()
# Index of missing person
clinical_df_clean[clinical_df_clean['Gender'].isnull()].index.tolist()

# Dealing with anomalous value in BMI (missing sample 199 & 155 - anomaly)
clinical_df_clean['BMI'].describe() # BMI = 2625? - Remove or replace with 26.25?
pd.options.mode.chained_assignment = None  # default='warn'
clinical_df_clean['BMI'][clinical_df_clean['BMI']>50]= np.nan

# Urban/Rural - complete
clinical_df_clean['Urban/Rural'].describe()

# Smoker - Yes and Y - change to all Yes (missing sample 199)
clinical_df_clean['Smoker (Y/N) & no.'].value_counts()
clinical_df_clean['Smoker (Y/N) & no.'][clinical_df_clean['Smoker (Y/N) & no.']=='Y'] = 'Yes'

# Lots of different reasons - most freq = 'Not hospitalised' (154), then 'Gastrointestinal' (17). 
# Some include mutliple diagnoses.
clinical_df_clean['Reason for hospital admission'].value_counts()
clinical_df_clean['Reason for hospital admission'][clinical_df_clean['Reason for hospital admission']=='Not Known']= None


# Lots of different co-morbidites - most freq = 'No', then 'Epilepsy' - Could just change to yes or no?
display(clinical_df_clean['Co-morbidities'].value_counts())
clinical_df_clean[clinical_df_clean['BMI'].isnull()].index.tolist()

/var/folders/hc/c0m0nxk93qb_ggmlvbk7wtbh0000gn/T/ipykernel_48760/485087828.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clinical_df_clean['Age'][clinical_df_clean['Age']<18]=np.nan


No                                         196
Epilepsy                                    10
Hypertention                                 3
High choles                                  2
communicating hydrocephalus                  1
Hypothyroidism                               1
Hypertention, Drawsines, Disorientation      1
Diabetes                                     1
TB                                           1
Aplastic A/high BP                           1
IDDM                                         1
Known case of hypertention                   1
Paralysis befor 12 years                     1
Name: Co-morbidities, dtype: int64

[156, 200]

# ML setup - feature & targey

In [8]:
# import variaous sublibraries from sklearn
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import  compose, pipeline, preprocessing, linear_model, tree, ensemble, model_selection, metrics, multiclass, svm
from sklearn.impute import SimpleImputer, KNNImputer

# Feature variables
features=['Age', 'Gender', 'BMI', 'Urban/Rural','Toilet (Y/N)', 'Hand soap', 'Smoker (Y/N) & no.',
   'Reason for hospital admission', 'Co-morbidities', 'Antibiotics (Y/N)']

# use_variables=list(glycome_clean_df.columns)

### Fearture - All variables - Early integration
# Concat cleaned datasets
df_cleaned = res = pd.concat([clinical_df_clean, glycome_df, scfa_df], axis=1).sort_index()
# Getting only features
X = df_cleaned.iloc[:,1:]

# Target variable
y = df_cleaned['Diarrheal/ Non Diarrheal']



#################
# Changing 'Reason for hospital admission' & 'Co-morbidities' to binary for simplicity
X_simple = X.copy()
# 'Reason for hospital admission'
X_simple['Reason for hospital admission'] = pd.Series(np.where(X_simple['Reason for hospital admission'] == 'Not Hospitalised', 'Not Hospitalised', 'Yes'),
          X_simple.index)
X_simple.loc[[220, 218, 216, 198, 74], 'Reason for hospital admission'] = np.nan
# 'Co-morbidities'
X_simple['Co-morbidities'] = pd.Series(np.where(X_simple['Co-morbidities'] == 'No', 'No', 'Yes'),
          X_simple.index)

# Encoding variables into binary 
X_simple['Gender'] = X_simple.Gender.map({'M':0, 'F':1})
X_simple['Urban/Rural'] = X_simple['Urban/Rural'].map({'Urban':0, 'Rural':1})
X_simple['Toilet (Y/N)'] = X_simple['Toilet (Y/N)'].map({'No':0, 'Yes':1})
X_simple['Hand soap'] = X_simple['Hand soap'].map({'No':0, 'Yes':1})
X_simple['Smoker (Y/N) & no.'] = X_simple['Smoker (Y/N) & no.'].map({'No':0, 'Yes':1})
X_simple['Reason for hospital admission'] = X_simple['Reason for hospital admission'].map({'Not Hospitalised':0, 'Yes':1})
X_simple['Co-morbidities'] =X_simple['Co-morbidities'].map({'No':0, 'Yes':1})
X_simple['Antibiotics (Y/N)'] = X_simple['Antibiotics (Y/N)'].map({'No':0, 'Yes':1})



###################
# Splitting data into test and train (pre-specified by Waleed split)
# Test
test_X = X_simple.loc[[1,4,9,19,23,29,30,36,43,44,64,67,83,86,87,105,108,115,119,127,130,131,133,134,140,153,155,
                           158,159,174,175,178,180,185,192,194,207,208,213,214,215,221],:]
test_y = y.loc[y.index.isin(test_X.index)]

# Train
train_X = X_simple.loc[~X.index.isin(test_X.index)]
train_y = y.loc[~y.index.isin(test_X.index)]

# Changing y values to 0 and 1
train_y = train_y.replace({'Diarrheal' : 1, 'Non Diarrheal' : 0})
test_y = test_y.replace({'Diarrheal' : 1, 'Non Diarrheal' : 0})

# Pipeline - preprocessor & classifier

In [9]:
# Pre-processing pipeline

# control, which variables to use. 
use_variables = [
    'age', 
    'sex', 
    'cp', 
    'trestbps', 
    'chol', 
    'fbs', 
    'restecg', 
    'thalach',
    'exang', 
    'oldpeak', 
    'slope', 
    'ca', 
    'thal'
]


# Numeric - All
numeric_features = list(glycome_df.columns) + list(scfa_df.columns)
# numeric_features = [x for x in numeric_features if x in use_variables]
numeric_transformer = pipeline.Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', preprocessing.StandardScaler())
])


### Count
count_features = ['Age']
count_transformer = pipeline.Pipeline(steps=[
                                      ('imputer', KNNImputer(n_neighbors=1)),
                                      ('scaler', preprocessing.StandardScaler())
])
# count_features = [x for x in count_features if x in use_variables]


### Categorical
categorical_features = ['Gender',
                        'Urban/Rural',
                        'Toilet (Y/N)',
                        'Hand soap',
                        'Smoker (Y/N) & no.',
                        'Reason for hospital admission',
                        'Co-morbidities',
                        'Antibiotics (Y/N)'
]

# categorical_features = [x for x in categorical_features if x in use_variables]
categorical_transformer = KNNImputer(n_neighbors=3)



# Preprocessor - simple All
allvar_preprocessor = compose.ColumnTransformer(
    transformers=[
        ('count', count_transformer, count_features),
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])




In [10]:
RandomForest = pipeline.Pipeline(steps=[
            ('preprocessor', allvar_preprocessor),
            ('classifier', (ensemble.RandomForestClassifier(random_state = 1)))
])

RandomForest

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('count',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=1)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Low-branching glycans (LB)',
                                            

# Parameters for grid search

In [11]:
# Grid search for best parameter
params = {
    'classifier__n_estimators':             [5,10,20,30],
    'classifier__criterion':                ['gini','entropy'],
    'classifier__max_depth':                [4,5,7],
    'classifier__min_samples_split':        [2,4,5,10],
}

# Grid search and ROC curve setup

In [12]:
def run_grid_search(X_kf, y_kf,cv_object,cv_inner=5,scoring='roc_auc',out_dir='.',viz_plot=False):
    
    classes     = sorted(list(y_kf.unique()))
    len_classes = len(classes)
    y_bin       = preprocessing.label_binarize(y_kf, classes=classes)
    #patch needed as label_binarize returns single valued lists for n = 2
    if len_classes == 2:
        bin_map = {0:[1,0],1:[0,1]}
        y_bin = np.array(y_kf.map(bin_map).to_list())

#     y_bin
    n_classes = 1
    print(n_classes)


    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)

    grid_search = model_selection.GridSearchCV(RandomForest, 
                           params, 
                           cv=5, 
                           scoring='roc_auc')

    result_hash = {
    }

    result_hash['n_classes'] = n_classes


    for enumerated_i, (train, test) in enumerate(cv_object.split(X_kf, y_kf)):
        e_i = enumerated_i
        result_hash[e_i] = {}

        
        result_hash[e_i]['train']     = train
        result_hash[e_i]['test']      = test
        result_hash[e_i]['n_classes'] = n_classes
        
        
        grid_search.fit(X_kf.iloc[train],  y_kf.iloc[train])

        # print out the best model after this
        print('Best params : {}'.format(grid_search.best_params_))
        result_hash[e_i]['best_params_'] = grid_search.best_params_

        # use best clasifier from internal grid
        classifier = grid_search.best_estimator_
        result_hash[e_i]['best_estimator_'] = grid_search.best_estimator_
        
        
# #         # Feature importance
# #         grid_search.best_estimator_[1][1].feature_importances_

# #         Printing feature importance
#         for feature_name,feature_importance in zip(X_kf.iloc[train].columns.values,grid_search.best_estimator_[1][1].feature_importances_):
#             if feature_importance > 0.0:
#                 print('{:20s}:{:3.4f}'.format(feature_name,feature_importance))

        # Feature importance with df
        df_importance = pd.DataFrame(list(zip(X_kf.iloc[train].columns.values,grid_search.best_estimator_[1][1].feature_importances_)),columns=['column_name','feature_importance'])
        df_importance = df_importance.set_index(['column_name'])
        df_importance.sort_values(['feature_importance'],ascending=False,inplace=True)
        
        
        # we need prediction probabilities for computing a ROC curve
        y_proba = classifier.predict_proba(X_kf.iloc[test])

        y_test  = y_kf.iloc[test]
        

        result_hash[e_i]['X_test']       = X_kf.iloc[test]
        result_hash[e_i]['y_test']       = y_test
        result_hash[e_i]['y_test_proba'] = y_proba[:,1]
        
        
        result_hash[e_i]['fpr']        = {}
        result_hash[e_i]['tpr']        = {}
        result_hash[e_i]['thrs']       = {}
        result_hash[e_i]['auc']        = {}
        result_hash[e_i]['feat_impor'] = {}

 
        result_hash[e_i]['fpr'][0], result_hash[e_i]['tpr'][0],result_hash[e_i]['thrs'][0] = metrics.roc_curve(y_test, y_proba[:,1])

        result_hash[e_i]['auc'][0] = metrics.auc(result_hash[e_i]['fpr'][0], result_hash[e_i]['tpr'][0])
        
        result_hash[e_i]['feat_impor'] = df_importance

        print('auc', result_hash[e_i]['auc'])

    return result_hash

#0.23-patch
#required as 0.23 does not have RocCurveDisplay.from_predictions implemented
def RocCurveDisplay_from_predictions(
#        cls,
        y_true,
        y_pred,
        *,
        sample_weight=None,
        drop_intermediate=True,
        pos_label=None,
        name=None,
        ax=None,
        **kwargs,
    ):         
    
    fpr, tpr, _ = metrics.roc_curve(
            y_true,
            y_pred,
            pos_label=pos_label,
            sample_weight=sample_weight,
            drop_intermediate=drop_intermediate,
    )
    roc_auc = metrics.auc(fpr, tpr)

    name = "Classifier" if name is None else name
    #pos_label = _check_pos_label_consistency(pos_label, y_true)

    viz = metrics.RocCurveDisplay(
            fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name=name, # pos_label=pos_label
    )

    return viz.plot(ax=ax, name=name, **kwargs)


# K-fold cross validation and ROC/Feature importance charts

In [13]:
import matplotlib.pyplot as plt  
%matplotlib inline

import seaborn as sns

# import variaous sublibraries from sklearn
from sklearn import  compose, pipeline, preprocessing, linear_model, tree, ensemble, model_selection, metrics, multiclass, svm


n_splits = 6
cv_object = model_selection.StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=0)

X_kf = X_simple.copy() # and (x not in missing_cols)]
y_kf = y.replace({'Diarrheal' : 1, 'Non Diarrheal' : 0})


classes   = sorted(list(y.unique()),)

res = {}

clfs = {
    'RandomForest':{
        'clf': RandomForest
    }
}

# for i in [x for x in clfs.keys()]:
#     print(i)

result_hash = run_grid_search(X_kf, 
                y_kf,
                cv_object,
                cv_inner=5,
                scoring='roc_auc',
        )
res['Random Forest']  = result_hash

n_classes = result_hash['n_classes']
print(n_classes)


## Mean feature importance
df_mean_feat_impor = pd.DataFrame(list(zip(X_kf.iloc[train].columns.values,[0]*34)),
                               columns=['column_name','feature_importance']).set_index(['column_name'])

for k in [x for x in result_hash if x not in ['n_classes']]:
    
    df_mean_feat_impor = df_mean_feat_impor + result_hash[k]['feat_impor']

df_mean_feat_impor['feature_importance'] = df_mean_feat_impor['feature_importance']/n_splits
df_mean_feat_impor.sort_values(['feature_importance'],ascending=False,inplace=True)


## plotting feature importance
plt.figure(figsize=(20,10))
import seaborn as sns
sns.barplot(x='column_name',y='feature_importance',data=df_mean_feat_impor.iloc[0:10,:].reset_index(),palette='muted')
ticks_information = plt.xticks(rotation=65)
display()



fig, ax = plt.subplots(1,n_classes,figsize=((n_classes)*11,10))

## plot roc curves for each class
for n_class in range(n_classes):
    print(n_class)
    det_curve = None

    mean_fpr = np.linspace(0, 1, 100)
    tprs = []
    aucs = []

    for k in [x for x in result_hash if x not in ['n_classes']]:

        local_ax = None
        if det_curve:
            local_ax = det_curve.ax_
        else:
            local_ax = ax

        #det_curve = metrics.RocCurveDisplay.from_predictions(
        det_curve = RocCurveDisplay_from_predictions(
                result_hash[k]['y_test'],
                result_hash[k]['y_test_proba'],
                name='ROC fold {}'.format(str(k+1).zfill(2)),
                pos_label = 1,
                alpha=0.3, 
                lw=1, 
                ax=local_ax,
            )
        local_ax = det_curve.ax_

        interp_tpr = np.interp(mean_fpr, det_curve.fpr, det_curve.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(det_curve.roc_auc)

    class_ax = det_curve.ax_
    class_ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',label='default', alpha=.8)


    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = metrics.auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)

    class_ax.plot(mean_fpr, 
        mean_tpr, 
        color='b',
        label=r'Mean ROC (AUC = {:0.3f} $\pm$ {:0.3f})'.format(mean_auc, std_auc),
        lw=2, 
        alpha=.8)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    class_ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                label=r'$\pm$ 1 std. dev.')

    class_ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
        title='Receiver operating characteristic - Classifiers: {}s (class={})'.format('Random Forest',classes[1]))
    class_ax.legend(loc="lower right")
    class_ax.set_xlabel('False Positive Rate')
    class_ax.set_ylabel('True Positive Rate')


1
Best params : {'classifier__criterion': 'gini', 'classifier__max_depth': 4, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 30}
auc {0: 0.9711538461538461}
Best params : {'classifier__criterion': 'entropy', 'classifier__max_depth': 5, 'classifier__min_samples_split': 10, 'classifier__n_estimators': 30}
auc {0: 0.9935897435897436}
Best params : {'classifier__criterion': 'gini', 'classifier__max_depth': 7, 'classifier__min_samples_split': 10, 'classifier__n_estimators': 30}
auc {0: 0.9711538461538461}
Best params : {'classifier__criterion': 'gini', 'classifier__max_depth': 5, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 20}
auc {0: 0.9813664596273292}
Best params : {'classifier__criterion': 'entropy', 'classifier__max_depth': 7, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 30}
auc {0: 0.9596273291925466}
Best params : {'classifier__criterion': 'gini', 'classifier__max_depth': 5, 'classifier__min_samples_split': 10, 'classifier__n_es

NameError: name 'train' is not defined

# Trial & error code

In [141]:
train= list(range(0,34,1))

fake = np.array([0]*34)

mean_feat_impor = pd.DataFrame(list(zip(X_kf.iloc[train].columns.values,[0]*34)),columns=['column_name','feature_importance']).set_index(['column_name'])

# mean_feat_impor + result_hash[0]['feat_impor']